In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.model_selection import train_test_split
import cv2
 

BASEPATH = '../input/d/wangps/aoi-detection'
TRAINIMG =  '../input/d/wangps/aoi-detection/train_images/train_images'
TESTIMG =  '../input/d/wangps/aoi-detection/test_images/test_images'
train_df = pd.read_csv(os.path.join(BASEPATH, 'train.csv'))
test_df = pd.read_csv(os.path.join(BASEPATH, 'test.csv'))
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)
print(train_df.shape)
print(val_df.shape)
print(test_df.shape)

In [2]:
class AOIdataset(object):
    def __init__(self, dataframe, image_dir, transforms=None, test=False):
        super().__init__()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms
        self.test = test

    def __getitem__(self, idx: int):
        id = self.df.iloc[idx]['ID']
        img = cv2.imread(os.path.join(self.image_dir, id))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
        if self.transforms:
            img = self.transforms(img)
        if not self.test:
            label = self.df.iloc[idx]['Label']
            label = torch.tensor(label, dtype=torch.long)
            return id, img, label
        else:
            return id, img
    def __len__(self):
        return len(self.df)
            

In [3]:
from torchvision import transforms
def getTransforms():
    originTransform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Resize((256,256)),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                ])
    return originTransform


In [4]:
from torch.utils.data import DataLoader,Dataset
train_dataset = AOIdataset(train_df, TRAINIMG, transforms=getTransforms())
val_dataset = AOIdataset(val_df, TRAINIMG, transforms=getTransforms())
test_dataset = AOIdataset(test_df, TESTIMG, transforms=getTransforms(), test=True)

trainDataLoader = DataLoader(train_dataset,batch_size=128,shuffle=True,num_workers=4,drop_last=True)
validDataLoader = DataLoader(val_dataset,batch_size=128,shuffle=False,num_workers=4)
testDataLoader = DataLoader(test_dataset,batch_size=128,shuffle=False,num_workers=4)

In [5]:
import torch
import torchvision.models as models

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.resnet18(pretrained=True) 
model = model.to(device)
criterion  = torch.nn.CrossEntropyLoss()
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.01,momentum=0.9, weight_decay=0.0005,nesterov=True)


In [6]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score
def train_one_epoch(model, dataloader, criterion, optimizer):
    model.train()
    totalloss=0
    totalacc=0
    with tqdm(dataloader,unit='batch',desc='Train') as tqdm_loader:
        for idx, (imgid, img,label) in enumerate(tqdm_loader):
            img = img.to(device)
            label = label.to(device)

            pred = model(img).to(device)

            loss = criterion(pred,label)
            
            pred = pred.cpu().detach().argmax(dim=1)

            optimizer.zero_grad()
            loss.backward()           
            optimizer.step()
            
            nowloss = loss.detach().item()
            totalloss +=nowloss
            
            acc = accuracy_score(pred,label.cpu())
            totalacc +=acc
            
            tqdm_loader.set_postfix(loss=nowloss,avgloss=totalloss/(idx+1),avgACC=totalacc/(idx+1) )

@torch.no_grad()
def evaluate(model, dataloader, criterion):
    model.eval()
    totalloss=0
    bestloss=10
    totalacc=0
    with torch.no_grad():
        with tqdm(dataloader,unit='batch',desc='Valid') as tqdm_loader:
            for idx, (imgid, img,label) in enumerate(tqdm_loader):

                img = img.to(device)
                label = label.to(device)

                pred = model(img)
                
                loss = criterion(pred,label)
                
                pred = pred.cpu().detach().argmax(dim=1)
                
                nowloss = loss.detach().item()
                totalloss +=nowloss
                
                acc = accuracy_score(pred,label.cpu())
                totalacc +=acc

                tqdm_loader.set_postfix(loss=nowloss,avgloss=totalloss/(idx+1),avgACC=totalacc/(idx+1) )
                
            avgloss = totalloss/len(tqdm_loader)
            if avgloss<=bestloss:
                bestloss = avgloss
                torch.save(model.state_dict(),'../best_model.pkl')


In [7]:
for epoch in range(10):
    print('\nEpoch {}'.format(epoch))
    train_one_epoch(model,trainDataLoader,criterion,optimizer)
    evaluate(model,validDataLoader,criterion)

In [9]:
model.load_state_dict(torch.load('../best_model.pkl'))

def test(model, dataloader):
    model.eval()
    pred_list =[]
    pred_id_list=[]
    with torch.no_grad():
        with tqdm(dataloader,unit='batch',desc='Test') as tqdm_loader:
            for idx, (imgid, img) in enumerate(tqdm_loader):

                img = img.to(device)
                pred = model(img).detach().cpu().argmax(dim=1)

                pred_list.append(pred)
                pred_id_list.append(imgid)
    pred_list = np.concatenate(pred_list,axis=0)
    pred_id_list =  np.concatenate(pred_id_list,axis=0)
    return pred_list,pred_id_list
pred_list,pred_id_list = test(model,testDataLoader)


In [15]:
submit_df  = pd.DataFrame({'ID':pred_id_list,'Label':pred_list})
# submit_df['ID'] = pred_id_list
# submit_df['Label']=pred_list
submit_df.to_csv('submit.csv',index=False)
print(submit_df.head(10))
